# Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
import os
import time

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
#XGBoost
import xgboost as xgb

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Import

In [ ]:
# Define the directory where your data is stored
save_dir = save_dir = '/content/drive/My Drive/XAI/processedData'

In [ ]:
# Load train data
train_data_ethiopia = pd.read_csv(os.path.join(save_dir, 'train_data_ethiopia.csv'))
X_train = train_data_ethiopia.iloc[:, :-1]
y_train = train_data_ethiopia.iloc[:, -1]

print("Shape - X_train: ", X_train.shape)
print("Shape - y_train: ", y_train.shape)

Shape - X_train:  (6931, 23)
Shape - y_train:  (6931,)


In [ ]:
# Load test data
test_data_ethiopia = pd.read_csv(os.path.join(save_dir, 'test_data_ethiopia.csv'))
X_test = test_data_ethiopia.iloc[:, :-1]
y_test = test_data_ethiopia.iloc[:, -1]

print("Shape - X_test: ", X_test.shape)
print("Shape - y_test: ", y_test.shape)

Shape - X_test:  (1733, 23)
Shape - y_test:  (1733,)


In [ ]:
# Load train (smote) data
train_data_smote = pd.read_csv(os.path.join(save_dir, 'train_data_smote.csv'))
X_train_smote = train_data_smote.iloc[:, :-1]
y_train_smote = train_data_smote.iloc[:, -1]

print("Shape - X_train_smote: ", X_train_smote.shape)
print("Shape - y_train_smote: ", y_train_smote.shape)

Shape - X_train_smote:  (17625, 23)
Shape - y_train_smote:  (17625,)


# Keras Model Definition

In [ ]:
def keras_model_implement(trainX, trainY, testX, testY):
  inputs = tf.keras.Input(shape=(trainX.shape[1],))
  x = tf.keras.layers.Dense(64, activation='relu')(inputs)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)

  model.compile(
      optimizer='adam',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  batch_size = 40
  epochs = 100

  history = model.fit(
      trainX,
      trainY,
      validation_data=(testX, testY),
      batch_size=batch_size,
      epochs=epochs,
      callbacks=[
          tf.keras.callbacks.ReduceLROnPlateau(),
          tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              patience=3,
              restore_best_weights=True
          )
      ]
  )

  return model, history

# Keras - Normal

In [ ]:
model_keras_normal, history_normal = keras_model_implement(X_train, y_train, X_test, y_test)

Epoch 1/100
174/174 [==============================] - 1s 3ms/step - loss: 0.9221 - accuracy: 0.7969 - val_loss: 0.5888 - val_accuracy: 0.8523 - lr: 0.0010
Epoch 2/100
174/174 [==============================] - 0s 2ms/step - loss: 0.5667 - accuracy: 0.8375 - val_loss: 0.6721 - val_accuracy: 0.7825 - lr: 0.0010
Epoch 3/100
174/174 [==============================] - 0s 2ms/step - loss: 0.6389 - accuracy: 0.8244 - val_loss: 0.6509 - val_accuracy: 0.8534 - lr: 0.0010
Epoch 4/100
174/174 [==============================] - 0s 3ms/step - loss: 0.5346 - accuracy: 0.8390 - val_loss: 0.6521 - val_accuracy: 0.6855 - lr: 0.0010


In [ ]:
print("Test Accuracy | Keras (Normal):", model_keras_normal.evaluate(X_test, y_test, verbose=0)[1])

Test Accuracy | Keras (Normal): 0.8522793054580688


In [ ]:
# Save the model
model_keras_normal.save("/content/drive/My Drive/XAI/models/kerasEthiopia")

# Keras - SMOTE

In [ ]:
model_keras_smote, history_smote = keras_model_implement(X_train_smote, y_train_smote, X_test, y_test)

Epoch 1/100
441/441 [==============================] - 2s 2ms/step - loss: 1.4803 - accuracy: 0.3894 - val_loss: 0.6648 - val_accuracy: 0.8032 - lr: 0.0010
Epoch 2/100
441/441 [==============================] - 1s 2ms/step - loss: 1.1455 - accuracy: 0.4514 - val_loss: 0.9464 - val_accuracy: 0.5591 - lr: 0.0010
Epoch 3/100
441/441 [==============================] - 1s 2ms/step - loss: 1.1209 - accuracy: 0.4682 - val_loss: 2.2574 - val_accuracy: 0.1558 - lr: 0.0010
Epoch 4/100
441/441 [==============================] - 1s 2ms/step - loss: 0.9848 - accuracy: 0.5151 - val_loss: 0.7554 - val_accuracy: 0.6965 - lr: 0.0010


In [ ]:
print("Test Accuracy:", model_keras_smote.evaluate(X_test, y_test, verbose=0)[1])

Test Accuracy: 0.803231418132782


In [ ]:
# Save the model
model_keras_smote.save("/content/drive/My Drive/XAI/models/kerasSmoteEthiopia")

# XGBoost - Normal

In [ ]:
model_xgboost_normal = xgb.XGBClassifier()

In [ ]:
start_time = time.time()
model_xgboost_normal.fit(X_train, y_train, verbose=True)
end_time = time.time()

elapsed_time = end_time - start_time
print("Time taken for model fitting:", elapsed_time, "seconds")

Time taken for model fitting: 2.7572500705718994 seconds


In [ ]:
# Make predictions on the test set
y_pred = model_xgboost_normal.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8528563185227929


In [ ]:
import pickle

# Save the model
with open("/content/drive/My Drive/XAI/models/xgboostEthiopia.pkl", "wb") as f:
    pickle.dump(model_xgboost_normal, f)

# XGBoost - SMOTE

In [ ]:
model_xgboost_smote = xgb.XGBClassifier()

In [ ]:
start_time = time.time()
model_xgboost_smote.fit(X_train_smote, y_train_smote, verbose=True)
end_time = time.time()

elapsed_time = end_time - start_time
print("Time taken for model fitting:", elapsed_time, "seconds")

Time taken for model fitting: 0.9156949520111084 seconds


In [ ]:
# Make predictions on the test set
y_pred = model_xgboost_smote.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8147720715522215


In [ ]:
# Save the model
with open("/content/drive/My Drive/XAI/models/xgboostSmoteEthiopia.pkl", "wb") as f:
    pickle.dump(model_xgboost_smote, f)